In [ ]:
import numpy
import random
import math
import sys
import os
import time
import collections
from collections import deque
import hashlib

numpy.set_printoptions(threshold=sys.maxsize)
numpy.set_printoptions(linewidth=300)

#Global Master Seed
MASTER_PLANET_SEED = "2D9C4F1E5A8B7C6D0E1F2A3B4C5D6E7F8A9B0C1D2E3F4A5B6C7D8E9F0A1B2C3D"

# --- Global Constants for Traversal Cost Analysis ---
MIN_SOIL_DEPTH = 0.1
MAX_ELEVATION = 5
MAX_WATER_LEVEL = 0.1
MAX_MOISTURE = 0.1

DEFAULT_TRAVERSAL_COST = 1.0
IMPASSABLE_COST = math.inf
TRAVERSE_LOGIX = [[-1,0],[0,1],[1,0],[0,-1]]

# --- Grid Generation Parameters (ADJUSTED AGAIN FOR CONNECTIVITY) ---
PLANET_GRID_SIZE = 30
MAX_GENERATION_ATTEMPTS = 20 # Increased attempts to find a valid grid

# --- Cluster Growth Parameters ---
# FlatGround (Landmasses - Made significantly MORE dominant for connectivity)
PLATEAU_SEEDS = 10 # More initial landmass seeds
PLATEAU_GROW_ITERATIONS = 10 # Drastically more iterations for larger, more connected islands
PLATEAU_GROW_PROB = 0.3756 # Very high probability for FlatGround to spread (almost guaranteed to spread to EmptySpace)

# --- Colors ---
BOLD = "\033[1m"
RESET = "\033[0m"
GREEN = "\033[92m"   # For success messages
YELLOW = "\033[33m"  # For warnings/status updates
BLUE = "\033[94m"   # For general info
MAGENTA = "\033[95m" # For analytical data
CYAN = "\033[96m"    # For vehicle status
RED = "\033[91m"    # For critical warnings/impassable
GRAY = "\033[90m"   # For subtle background info

# Defining Base Terrain Class
class TerrainObject :
    def __init__(self, name, desc, geoProp, stability, obstacle, traversable):
        self.name = name
        self.desc = desc
        self.geoProp = geoProp if geoProp is not None else {"soilDepth": 1,"elevation": 0,"waterLevel": 0, "soilMoisture": 0.1}
        self.stability = stability
        self.obstacle = obstacle
        self.traversable = traversable
    
    def __repr__(self):
        return f"{self.name}"
    
class plataeu(TerrainObject):
    def __init__(self):
        super().__init__(f"{YELLOW}\u2588{RESET}","Plateau: A vast land mass having abrupt elevation compared to plains", {"soilDepth": 1,"elevation": random.randint(MAX_ELEVATION, 50),"waterLevel": 0, "soilMoisture": 0.1}, 1, True, False)

class plains(TerrainObject):
    def __init__(self):
        super().__init__(f"{CYAN}\u25A0{RESET}","Plains: Cluster of a continent where various other TerrainObject coexists", {"soilDepth": 1,"elevation": random.randint(0, MAX_ELEVATION-1),"waterLevel": 0, "soilMoisture": 0.3}, 1, False, True)

# Function to handle grid generation and seed 
def generateGrid (PLANET_GRID_SIZE):
    gridSectorDelta = numpy.empty((PLANET_GRID_SIZE, PLANET_GRID_SIZE), dtype=object)
    for r in range(len(gridSectorDelta)):
        for c in range(len(gridSectorDelta)):
            gridSectorDelta[r][c] = plains()
    return gridSectorDelta

def clear_screen():
    # For Windows
    if os.name == 'nt':
        _ = os.system('cls')
    # For macOS and Linux (os.name is 'posix' for these)
    else:
        _ = os.system('clear')

def create_clustered_grid ():
    tempSpreadCell = collections.deque()
    activeSpreadCell = collections.deque()

    for _ in range(PLATEAU_SEEDS):
        randRow = random.randint(0, len(gridSectorAlpha)-1)
        randCol = random.randint(0, len(gridSectorAlpha)-1)
        gridSectorAlpha[randRow][randCol] = plataeu()
        activeSpreadCell.append([randRow, randCol])

    #Seeding Plataeu
    for growCycle in range(PLATEAU_GROW_ITERATIONS): # Iterates for 10x - 0
        for i in range(len(activeSpreadCell)): # Iterates for len(deque) - [43, 12]
            row = activeSpreadCell[i][0] # 43
            col = activeSpreadCell[i][1] # 12
            for nrow, ncol in TRAVERSE_LOGIX:
                moveR = row + nrow # 42
                moveC = col + ncol # 12
                actualProbability = random.uniform(0.0, 1.0)
                if (moveR >= 0 and moveR < gridSectorAlpha.shape[0] and moveC >= 0 and moveC < gridSectorAlpha.shape[1]) and (isinstance(gridSectorAlpha[moveR][moveC], plains) and (actualProbability>PLATEAU_GROW_PROB)):
                    gridSectorAlpha[moveR][moveC] = plataeu()
                    tempSpreadCell.append([moveR, moveC])
                for r in range(gridSectorAlpha.shape[0]):
                    for c in range(gridSectorAlpha.shape[1]):
                        print(gridSectorAlpha[r][c].__repr__(), end=" ")
                    print()
            #clear_screen()
        activeSpreadCell.clear()
        activeSpreadCell = collections.deque(tempSpreadCell)
        tempSpreadCell.clear()
    return gridSectorAlpha

def displayGrid (gridSectorAlpha):
    for r in range(gridSectorAlpha.shape[0]):
        for c in range(gridSectorAlpha.shape[1]):
            print(gridSectorAlpha[r][c].__repr__(), end=" ")
        print()

if __name__ == "__main__":
    #Initiating global seed
    random.seed(MASTER_PLANET_SEED)
    print(f"{GREEN}","-"*20,f"Initializing World generation of {PLANET_GRID_SIZE} x {PLANET_GRID_SIZE}", "-"*20, f"{RESET}")
    time.sleep(1)
    print(f"WORLD_SEED_INITIALIZATION: MASTER_PLANET_SEED{MAGENTA}{MASTER_PLANET_SEED}{RESET}")
    time.sleep(1)
    print(f"WORLD_SEED_INITIALIZATION: Applying UTF-8 Encoding to MASTER_PLANET_SEED ")
    seed_bytes = MASTER_PLANET_SEED.encode('utf-8')
    time.sleep(1)
    print(f"WORLD_SEED_INITIALIZATION: Processing SHA256-Hashing MASTER_PLANET_SEED")
    hash_digest = hashlib.sha256(seed_bytes).digest()
    time.sleep(1)
    print(f"WORLD_SEED_INITIALIZATION: Data type conversion to handle BIGINT {RESET}")
    numpy_seed_int = int.from_bytes(hash_digest[:4], 'big')
    time.sleep(1)
    print(f"WORLD_SEED_INITIALIZATION: MASTER_PLANET_SEED Initialized Successfully{RESET}")
    numpy.random.seed(numpy_seed_int)
    time.sleep(4)
    print(f"GENERATING_GRID: Attemptnig to create world using initialized seed{RESET}")
    gridSectorAlpha = generateGrid(PLANET_GRID_SIZE)
    time.sleep(2)
    print(f"GENERATING_GRID: Grid creation successfull{RESET}")
    time.sleep(4)
    print(f"GENERATING_CLUSTERS: Creating water, plains, plataeu etc...{RESET}")
    gridSectorAlpha = create_clustered_grid()
    time.sleep(1)
    print(f"GENERATING_CLUSTERS: World object addedd Successfully. World is Ready!{RESET}")
    time.sleep(2)
    print(f"STARTING_WORLD: Initializing Grid View{RESET}")
    time.sleep(5)
    print(f"{GREEN}-{RESET}"*50)
    displayGrid(gridSectorAlpha)


 -------------------- Initializing World generation of 30 x 30 -------------------- 
WORLD_SEED_INITIALIZATION: MASTER_PLANET_SEED2D9C4F1E5A8B7C6D0E1F2A3B4C5D6E7F8A9B0C1D2E3F4A5B6C7D8E9F0A1B2C3D
WORLD_SEED_INITIALIZATION: Applying UTF-8 Encoding to MASTER_PLANET_SEED 
WORLD_SEED_INITIALIZATION: Processing SHA256-Hashing MASTER_PLANET_SEED
WORLD_SEED_INITIALIZATION: Data type conversion to handle BIGINT 
WORLD_SEED_INITIALIZATION: MASTER_PLANET_SEED Initialized Successfully
GENERATING_GRID: Attemptnig to create world using initialized seed


In [ ]:
import pygame
import sys

pygame.init()
pygame.font.init()

SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Auto Print Log Demo")

# --- Colors ---
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GREEN = (0, 200, 0)
DARK_GRAY = (50, 50, 50)
LIGHT_GRAY = (150, 150, 150)

# --- Log Display Area Setup ---
log_rect = pygame.Rect(50, 50, SCREEN_WIDTH - 100, 200) # Rectangle for the log
log_padding = 10 # Padding inside the log rectangle

# --- Font Setup ---
font_size = 20
line_height = font_size + 2
log_font = pygame.font.SysFont("Consolas", font_size)

# Calculate how many lines can fit in our log_rect
MAX_DISPLAY_LINES = (log_rect.height - log_padding * 2) // line_height

# --- Message Storage ---
current_log_messages = [] # This list will hold the messages to display

# --- Auto-add Message Timer ---
MESSAGE_ADD_EVENT = pygame.USEREVENT + 1 # Custom event ID
pygame.time.set_timer(MESSAGE_ADD_EVENT, 2000) # Add a new message every 2 seconds

# --- Game Clock ---
clock = pygame.time.Clock()
FPS = 60

# --- Helper Function to Add Messages (simulates "auto print") ---
message_counter = 0
def add_new_log_message():
    global message_counter
    new_msg = f"Event {message_counter:03d}: System status nominal."
    current_log_messages.append(new_msg)
    message_counter += 1
    # If we have too many messages, remove the oldest one
    if len(current_log_messages) > MAX_DISPLAY_LINES:
        current_log_messages.pop(0) # Remove the first (oldest) message

# Add an initial message
add_new_log_message()

# --- Game Loop ---
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                running = False
            elif event.key == pygame.K_SPACE: # Manually add a message on spacebar
                add_new_log_message()
        elif event.type == MESSAGE_ADD_EVENT: # Custom event triggered by timer
            add_new_log_message()

    # --- Drawing ---
    screen.fill(BLACK) # Clear the screen

    # Draw log area background and border
    pygame.draw.rect(screen, DARK_GRAY, log_rect)
    pygame.draw.rect(screen, LIGHT_GRAY, log_rect, 2) # Border

    # Draw the current messages in the log
    for i, message in enumerate(current_log_messages):
        # Calculate Y position for each line, starting from the top of the log_rect
        # and moving downwards based on line index
        text_y = log_rect.y + log_padding + i * line_height
        text_x = log_rect.x + log_padding

        # Render the text
        text_surface = log_font.render(message, True, GREEN)

        # Blit (draw) the text surface
        screen.blit(text_surface, (text_x, text_y))

    pygame.display.flip()
    clock.tick(FPS)

pygame.quit()
sys.exit()

In [ ]:
GRV = [10,15,30,40,50]
ATM = [1,2,3,4,5]
pulesPower = [10,50,100,200,500]
pulseAfterboost = [1,2]
#totalResistansiveForce = GRV[0] * ATM[0]
#totalGeneratedForce = pulesPower[2] * pulseAfterboost[4]
#displacement = 1-((totalResistansiveForce - totalGenerateForce)  / 100)
velocity =  1 - (((GRV[0]*ATM[0]) - (pulesPower[4]*pulseAfterboost[1])) / 100)# pixels/sec
velocity

In [ ]:
dic = {

}

a = set()

a.add((1,2))
a.add((1,2))
a

    
